<a href="https://colab.research.google.com/github/dayoungcho/dacon/blob/main/%EC%A0%84%EB%A0%A5%EA%B1%B0%EB%9E%98%EC%86%8C_TFT%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

In [66]:
pip install pytimekr

  Preparing metadata (setup.py) ... done
  Created wheel for pytimekr: filename=pytimekr-0.1.0-py3-none-any.whl size=7923 sha256=627dd4372a332fd540bea320bf6111dc3e660e760dba3f3531dadeb505d5d036
  Stored in directory: /root/.cache/pip/wheels/df/bb/74/a8776ef7cbb5a05ebbcff3c386b16d8f5f08760a393ebec014
Successfully built pytimekr


In [89]:
import pandas as pd
import datetime as dt
import itertools
from pytimekr import pytimekr
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer

In [54]:
electrade_2013 = pd.read_csv('2013_2016 일별 시간대별 연료원별 전력거래량.csv', encoding='cp949')
electrade_2017 = pd.read_csv('2017~2020 일별 시간대별 연료원별 전력거래량.csv', encoding='cp949')
electrade_2021 = pd.read_csv('일별 시간대별 연료원별 거래량_20220430.csv', encoding='cp949')
electrade_2022 = pd.read_csv('한국전력거래소_연료원별 전력거래량_20231031.csv', encoding='cp949')
electrade_2023 = pd.read_csv('한국전력거래소_연료원별 전력거래량_20231231.csv', encoding='cp949')


In [55]:
# 데이터 병합

electrade_2013.columns = ['거래일', '거래시간', '연료원', '전력거래량']
electrade_2017.columns = ['거래일', '거래시간', '연료원', '전력거래량']
electrade_2021 = electrade_2021[['거래일', '거래시간', '연료원', '전력거래량']]
electrade_2022 = electrade_2022[['거래일', '거래시간', '연료원', '전력거래량(MWh)']]
electrade_2022.columns = ['거래일', '거래시간', '연료원', '전력거래량']
electrade_2023 = electrade_2023[['거래일', '거래시간', '연료원', '전력거래량(MWh)']]
electrade_2023.columns = ['거래일', '거래시간', '연료원', '전력거래량']

electrade = pd.concat([electrade_2013, electrade_2017, electrade_2021, electrade_2022, electrade_2023])

In [56]:
electrade

,거래일,거래시간,연료원,전력거래량
0,2013-01-01,0,원자력,16232.160050
1,2013-01-01,0,석탄,23638.589390
2,2013-01-01,0,LNG,13380.994560
3,2013-01-01,0,유류,697.044902
4,2013-01-01,0,양수,855.181596
...,...,...,...,...
30739,2023-12-31,20,기타,42.000000
30740,2023-12-31,21,기타,42.000000
30741,2023-12-31,22,기타,41.000000
30742,2023-12-31,23,기타,39.000000


In [ ]:
# 공휴일 리스트

holidays = []
for i in range(2013,2024):
  holidays.append(pytimekr.holidays(i))
holidays = list(itertools.chain(*holidays))

In [ ]:
electrade.loc[electrade['거래시간'] == 24, '거래시간'] = 0   # 자정 거래시간이 0이랑 24로 중복 표기되어있음 -> 0으로 통일
electrade.거래일 = pd.to_datetime(electrade.거래일)
electrade['월'] = electrade.거래일.dt.month.astype(str)
electrade['요일'] = electrade.거래일.dt.weekday.astype(str)
electrade['공휴일여부'] = electrade.거래일.isin(holidays)*1
electrade.columns = ['time_idx', 'hour', 'fuel_type', 'volume', 'month', 'day_of_week', 'is_holiday']
electrade

In [ ]:
# 모델 설정
max_prediction_length = 7 # 향후 7일을 예측 (decoder 길이)
max_encoder_length = 30   # 과거 30일을 참조 (encoder 길이)
training_cutoff = electrade.거래일.max() - pd.Timedelta(days=max_prediction_length) # 학습 데이터 자르는 시점

traing = TimeSeriesDataSet(
    electrade[lambda x: x.time_idx <= training_cutoff],   # 학습용 데이터만 필터링

    # ---기본 설정---
    time_idx = "time_idx",
    target = "volume",
    group_ids= ["fuel_type"],

    # ---시퀀스 길이 설정---
    min_encoder_length = max_encoder_length//2,     # 최소 참조 기간(가변적일 수 있음)
    max_encoder_length = max_encoder_length,        # 최대 참조 기간
    min_prediction_length = 1,                      # 최소 예측 기간
    max_prediction_length = max_prediction_length,  # 최대 예측 기간

    # ---변수 매핑(중요)---

    # 1. 정적 변수(static)
    static_categoricals = [''],

    # 2. 미래를 아는 변수(known inputs)
    time_varying_known_categoricals = [],
    time_varying_known_reals = [],   # 타겟도 포함시켜야함

    # 3. 과거만 아는 변수
)






In [46]:
# 일별, 시간별 LNG를 연료로 한 전력거래량
electrade_lng = electrade[electrade['연료원'] == 'LNG']
electrade_lng.reset_index(inplace=True, drop=True)
electrade_lng.drop('연료원', inplace=True,axis=1)
electrade_lng

/tmp/ipython-input-2914467080.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  electrade_lng.drop('연료원', inplace=True,axis=1)


,거래일,거래시간,전력거래량
0,2013-01-01,24,13380.994560
1,2013-01-01,1,12092.910380
2,2013-01-01,2,10736.999880
3,2013-01-01,3,9869.339046
4,2013-01-01,4,9123.019097
...,...,...,...
96403,2023-12-31,20,18996.000000
96404,2023-12-31,21,18404.000000
96405,2023-12-31,22,17872.000000
96406,2023-12-31,23,18306.000000


In [47]:
# 일별, 시간별 전력거래량
electrade_agg = electrade.groupby(['거래일','거래시간']).sum().reset_index()
electrade_agg.drop('연료원', inplace=True,axis=1)
electrade_agg

,거래일,거래시간,전력거래량
0,2013-01-01,1,53918.166773
1,2013-01-01,2,52207.046421
2,2013-01-01,3,51198.759433
3,2013-01-01,4,50523.000055
4,2013-01-01,5,50594.166019
...,...,...,...
96403,2023-12-31,20,60224.000000
96404,2023-12-31,21,59405.000000
96405,2023-12-31,22,58758.000000
96406,2023-12-31,23,59229.000000


In [48]:
# 일별 LNG를 연료로 한 전력거래량

electrade_dailylng = electrade_lng.groupby(['거래일']).sum().reset_index()
electrade_dailylng.drop('거래시간', inplace=True,axis=1)
electrade_dailylng

,거래일,전력거래량
0,2013-01-01,196479.142825
1,2013-01-02,376521.963524
2,2013-01-03,440858.162990
3,2013-01-04,450037.228490
4,2013-01-05,394665.561910
...,...,...
4012,2023-12-27,449425.000000
4013,2023-12-28,468120.000000
4014,2023-12-29,455049.000000
4015,2023-12-30,370363.000000


In [51]:
# 일별 총 전력거래량

electrade_agg_daily = electrade_agg.groupby('거래일').sum().reset_index()
electrade_agg_daily.drop('거래시간', inplace=True,axis=1)
electrade_agg_daily

,거래일,전력거래량
0,2013-01-01,1.208822e+06
1,2013-01-02,1.470775e+06
2,2013-01-03,1.569730e+06
3,2013-01-04,1.579830e+06
4,2013-01-05,1.455056e+06
...,...,...
4012,2023-12-27,1.685486e+06
4013,2023-12-28,1.674182e+06
4014,2023-12-29,1.614603e+06
4015,2023-12-30,1.457736e+06
